In [1]:
import sys
sys.path.append('..')
from src.models.pipeline import pipeline
import glob
import json
import os
import pandas as pd
import ast
from datetime import datetime
from shutil import copyfile
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs
from src.models.train_model import functionsBuilder
from src.models.audio import downloadAudioFromYoutube
import warnings
warnings.filterwarnings("ignore")

In [2]:
groundbase_dir = '../data/raw/groundbase'
transcripts_dir = os.path.join(groundbase_dir,'transcripts')
topic_dataset_path = os.path.join(groundbase_dir,'dataset.csv')
transcript_filespath = glob.glob(groundbase_dir + '/transcripts/*.json')

'''Read the transcript'''
transcripts_jsons = {}
for fl in transcript_filespath:
    with open(fl,encoding="utf8") as f:
        transcript =ast.literal_eval(f.read()) #json.load(f)
        vid = fl.split('\\')[-1].split('.')[0]
        #print(vid)
        transcripts_jsons[vid] = transcript
#print(transcripts_jsons)

'''Read the videos metadata to perform on them the segmentation'''
df_videos = pd.read_csv(topic_dataset_path)

''' Transfer topic shifts time to seconds units instead HH:MM:SS'''

def topic_shifts_seconds(topic_shifts):
    tp_shift_sec=[]
    for tp in topic_shifts:
        intervals = tp.split(':')
        seconds = int(intervals[2])
        minutes = int(intervals[1]) * 60
        hours = int(intervals[0]) * 60 *60
        tp_shift_sec.append(seconds + minutes + hours)
    return tp_shift_sec


for video_id in transcripts_jsons.keys():    
    df_videos.at[df_videos['video id'] == video_id,'topic shifts(ends)'] =\
    topic_shifts_seconds(\
                         df_videos[df_videos['video id'] == \
                                   video_id]['topic shifts(ends)'])

Divide the dataset into 3 groups ( sd,audio,image) as correspoding to each model.
we want to  prepare to use the k-folds. Each group is a Test set of the type of the model group (sd,audio,image), the rest is the training dataset. 

each model is comprise of 3 different pipelines:
    None filter
    median 2,2
    median 3,3


In [21]:
models_workflow = {
    #'sliding_window':[
        #'sliding_window-tfidf-cosine-None-spectral_clustering',
        #'sliding_window-tfidf-cosine-median_(2,2)-spectral_clustering'#,
        #"sliding_window-tfidf-cosine-median_(3,3)-spectral_clustering"
    #],
    'audio':[
        'audio-tfidf-cosine-None-spectral_clustering',
        'audio-tfidf-cosine-median_(2,2)-spectral_clustering',
        'audio-tfidf-cosine-median_(3,3)-spectral_clustering'
    ]#,
    #'image' : [
        
    #]
}
models = list(models_workflow.keys())
videos_ids = df_videos.drop_duplicates(subset=['video id'])['video id'].values.tolist()
#print(videos_ids)



In [11]:
'''Downloading wav files'''
if 'audio' in models:
    audio_path = '../data/raw/audio'
    for vid in videos_ids:
        if os.path.isfile(os.path.join(audio_path,vid+'.wav')) is False:
            url = (str(df_videos.loc[df_videos['video id'] == vid,'youtube link'].values.tolist()[0]))
            downloadAudioFromYoutube(url)

In [4]:
'''Finding Test dataset of each model'''

## probably k folds is not relevant here

'''step = int(len(videos_ids)/len(models))
test_datasets = [videos_ids[i:i + step] for i in range(0,len(videos_ids),step)]
# adding the modulu to the last group
if len(test_datasets[-1]) < len(models):
    test_datasets[-2] = test_datasets[-2] + test_datasets[-1]
    test_datasets.pop()
    
#print(test_datasets)
#print(len(test_datasets))
'''Building the Training and Test Set'''    
dataset = [
    {
        "train":[test_datasets[i] for i in range(len(test_datasets)) if i!=mdl_index],
        "test":test_datasets[mdl_index],
        'model':models[mdl_index]
    }
    for mdl_index in range(len(models))]'''

In [5]:
'''Creating the experiment uses'''
'''date_now = datetime.now().strftime("%d-%m-%Y %H-%M-%S")
exp_path = os.path.join('../models/bayesian_opt/experiment',date_now)
os.mkdir(exp_path)
file = 'parameters_bounds.json'
parameter_path = exp_path +'/' + file
copyfile('../models/bayesian_opt/poc/' + file,parameter_path)'''

'../models/bayesian_opt/experiment\\13-05-2020 12-40-08/parameters_bounds.json'

# *Configure now the parameters bounds as you wish*

In [25]:
'''Read the parameters bounds'''
with open(os.path.join('../models/bayesian_opt/experiment','parameters_bounds.json'),'r') as f:
    param_bounds = ast.literal_eval(f.read())
df_results = pd.read_csv('../data/processed/bayesian_opt/results.csv') #pd.DataFrame(columns=['video','max_target','workflow','params','param_bounds'])

In [16]:
df_results.head()
#param_bounds

,Unnamed: 0,video,max_target,workflow,params,param_bounds
0,0,MkiUBJcgdUY,0.222222,sliding_window-tfidf-cosine-None-spectral_clus...,"{'n_clusters': 14.502132028215444, 'sim_thresh...","{""window_size"": (20,150),""step_size"":(10,60),'..."
1,1,MkiUBJcgdUY,0.222222,sliding_window-tfidf-cosine-None-spectral_clus...,"{'n_clusters': 12.880535344902679, 'sim_thresh...","{""window_size"": (20,150),""step_size"":(10,60),'..."
2,2,MkiUBJcgdUY,0.444444,sliding_window-tfidf-cosine-None-spectral_clus...,"{'n_clusters': 15.53458736722668, 'sim_thresh'...","{""window_size"": (20,150),""step_size"":(10,60),'..."
3,3,MkiUBJcgdUY,0.277778,sliding_window-tfidf-cosine-None-spectral_clus...,"{'n_clusters': 12.0, 'sim_thresh': 0.300000010...","{""window_size"": (20,150),""step_size"":(10,60),'..."
4,4,MkiUBJcgdUY,0.222222,sliding_window-tfidf-cosine-None-spectral_clus...,"{'n_clusters': 13.425649132739782, 'sim_thresh...","{""window_size"": (20,150),""step_size"":(10,60),'..."


# Training the model

Train the models. Each video is running couple of times for bayesian opitmum.

In [29]:
for vid in videos_ids:
    print("running on video %s" %(vid))
    groundbase = df_videos.loc[df_videos['video id'] == vid,'topic shifts(ends)'].values.tolist()[:-1]
    tr = transcripts_jsons[vid]
    for model in models:
        #samples = [d["train"] for d in dataset if d[model] == model][0]
        for workflow_label in models_workflow[model]:
            print('Running workflow %s' %(workflow_label))
            #logger = JSONLogger(path=os.path.join(exp_path,('%s.json' %(workflow_label))))
            #optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)
            
            function_to_optimized = functionsBuilder(groundbase=groundbase,transcripts=tr).build_f_to_optimize(workflow_label)
            
            optimizer = BayesianOptimization(
                f=function_to_optimized,
                pbounds=param_bounds[workflow_label],
                verbose=2,
                random_state=1
                )
            
            optimizer.maximize(
                init_points = 6,
                n_iter = 14
            )
            
            print(optimizer.max)
            for res in optimizer.res:
                df_results = df_results.append({'video':vid,
                                                'max_target':res['target'],
                                                'workflow':workflow_label,
                                                'params':res['params'],
                                                'param_bounds':param_bounds[workflow_label]
                                               },ignore_index=True)
        

running on video MkiUBJcgdUY
Running workflow audio-tfidf-cosine-None-spectral_clustering
|   iter    |  target   | n_clus... | silenc... | sim_th... | slice_... | step_s... |
-------------------------------------------------------------------------------------
|  1        |  0.0      |  14.5     | -8.39     |  0.3      |  303.0    |  147.6    |
|  2        |  0.0      |  12.55    | -24.41    |  0.4037   |  397.4    |  539.3    |
|  3        |  0.0      |  14.52    | -9.443    |  0.3613   |  878.2    |  28.36    |
|  4        |  0.0      |  16.02    | -17.48    |  0.4676   |  141.2    |  198.9    |
|  5        |  0.0      |  16.8     | -0.9522   |  0.394    |  692.6    |  876.5    |
|  6        |  0.0      |  17.37    | -27.45    |  0.3117   |  170.7    |  878.3    |
|  7        |  0.0      |  16.54    | -27.84    |  0.5462   |  997.2    |  993.9    |
|  8        |  0.0      |  14.09    | -3.155    |  0.5466   |  1.496    |  977.0    |
|  9        |  0.0      |  16.18    | -27.74    | 

|  14       |  0.0      |  16.41    | -24.26    |  0.4545   |  8.301    |  19.18    |
|  15       |  0.0      |  12.52    | -0.168    |  0.4996   |  997.0    |  955.3    |
|  16       |  0.0      |  14.58    | -29.86    |  0.4871   |  988.5    |  16.3     |
|  17       |  0.0      |  13.18    | -0.2884   |  0.3072   |  21.9     |  988.9    |
|  18       |  0.0      |  13.53    | -1.319    |  0.4916   |  8.188    |  27.39    |
|  19       |  0.0      |  13.01    | -7.496    |  0.3558   |  490.5    |  717.1    |
|  20       |  0.0      |  12.04    | -26.98    |  0.4373   |  971.0    |  994.4    |
{'target': 0.0, 'params': {'n_clusters': 14.502132028215444, 'silence_threshold': -8.390265196735257, 'sim_thresh': 0.30003431244520345, 'slice_length': 303.0302400592079, 'step_size_audio': 147.60913492629592}}
Running workflow audio-tfidf-cosine-median_(2,2)-spectral_clustering
|   iter    |  target   | n_clus... | silenc... | sim_th... | slice_... | step_s... |
-------------------------------

|  3        |  0.0      |  13.35    | -9.443    |  0.3613   |  878.2    |  28.36    |
|  4        |  0.0      |  15.36    | -17.48    |  0.4676   |  141.2    |  198.9    |
|  5        |  0.0      |  16.41    | -0.9522   |  0.394    |  692.6    |  876.5    |
|  6        |  0.0      |  17.16    | -27.45    |  0.3117   |  170.7    |  878.3    |
|  7        |  0.0      |  16.05    | -27.84    |  0.5462   |  997.2    |  993.9    |
|  8        |  0.0      |  12.78    | -3.155    |  0.5466   |  1.496    |  977.0    |
|  9        |  0.0      |  15.58    | -27.74    |  0.3524   |  995.4    |  9.24     |
|  10       |  0.0      |  16.77    | -2.102    |  0.5967   |  2.982    |  12.13    |
|  11       |  0.0      |  13.0     | -1.156    |  0.4581   |  984.1    |  989.7    |
|  12       |  0.0      |  16.03    | -1.713    |  0.3766   |  983.2    |  2.261    |
|  13       |  0.0      |  10.95    | -29.56    |  0.4601   |  39.75    |  982.5    |
|  14       |  0.0      |  15.89    | -24.26    |  0.4

|  19       |  0.0      |  11.35    | -7.496    |  0.3558   |  490.5    |  717.1    |
|  20       |  0.0      |  10.05    | -26.98    |  0.4373   |  971.0    |  994.4    |
{'target': 0.0, 'params': {'n_clusters': 13.336176037620593, 'silence_threshold': -8.390265196735257, 'sim_thresh': 0.30003431244520345, 'slice_length': 303.0302400592079, 'step_size_audio': 147.60913492629592}}
Running workflow audio-tfidf-cosine-median_(3,3)-spectral_clustering
|   iter    |  target   | n_clus... | silenc... | sim_th... | slice_... | step_s... |
-------------------------------------------------------------------------------------
|  1        |  0.0      |  14.5     | -8.39     |  0.3      |  303.0    |  147.6    |
|  2        |  0.0      |  12.55    | -24.41    |  0.4037   |  397.4    |  539.3    |
|  3        |  0.0      |  14.52    | -9.443    |  0.3613   |  878.2    |  28.36    |
|  4        |  0.0      |  16.02    | -17.48    |  0.4676   |  141.2    |  198.9    |
|  5        |  0.0      |  16.8

|  8        |  0.0      |  14.09    | -3.155    |  0.5466   |  1.496    |  977.0    |
|  9        |  0.0      |  16.18    | -27.74    |  0.3524   |  995.4    |  9.24     |
|  10       |  0.0      |  17.08    | -2.102    |  0.5967   |  2.982    |  12.13    |
|  11       |  0.0      |  14.25    | -1.156    |  0.4581   |  984.1    |  989.7    |
|  12       |  0.0      |  16.52    | -1.713    |  0.3766   |  983.2    |  2.261    |
|  13       |  0.0      |  12.71    | -29.56    |  0.4601   |  39.75    |  982.5    |
|  14       |  0.0      |  16.41    | -24.26    |  0.4545   |  8.301    |  19.18    |
|  15       |  0.0      |  12.52    | -0.168    |  0.4996   |  997.0    |  955.3    |
|  16       |  0.0      |  14.58    | -29.86    |  0.4871   |  988.5    |  16.3     |
|  17       |  0.0      |  13.18    | -0.2884   |  0.3072   |  21.9     |  988.9    |
|  18       |  0.0      |  13.53    | -1.319    |  0.4916   |  8.188    |  27.39    |
|  19       |  0.0      |  13.01    | -7.496    |  0.3

...

|  11       |  0.0      |  14.25    | -1.156    |  0.4581   |  984.1    |  989.7    |
|  12       |  0.0      |  16.52    | -1.713    |  0.3766   |  983.2    |  2.261    |
|  13       |  0.0      |  12.71    | -29.56    |  0.4601   |  39.75    |  982.5    |
|  14       |  0.0      |  16.41    | -24.26    |  0.4545   |  8.301    |  19.18    |
|  15       |  0.0      |  12.52    | -0.168    |  0.4996   |  997.0    |  955.3    |
|  16       |  0.0      |  14.58    | -29.86    |  0.4871   |  988.5    |  16.3     |
|  17       |  0.0      |  13.18    | -0.2884   |  0.3072   |  21.9     |  988.9    |
|  18       |  0.0      |  13.53    | -1.319    |  0.4916   |  8.188    |  27.39    |
|  19       |  0.0      |  13.01    | -7.496    |  0.3558   |  490.5    |  717.1    |
|  20       |  0.0      |  12.04    | -26.98    |  0.4373   |  971.0    |  994.4    |
{'target': 0.0, 'params': {'n_clusters': 14.502132028215444, 'silence_threshold': -8.390265196735257, 'sim_thresh': 0.30003431244520345, 'sl

|  1        |  0.0      |  13.34    | -8.39     |  0.3      |  303.0    |  147.6    |
|  2        |  0.0      |  10.74    | -24.41    |  0.4037   |  397.4    |  539.3    |
|  3        |  0.0      |  13.35    | -9.443    |  0.3613   |  878.2    |  28.36    |
|  4        |  0.0      |  15.36    | -17.48    |  0.4676   |  141.2    |  198.9    |
|  5        |  0.0      |  16.41    | -0.9522   |  0.394    |  692.6    |  876.5    |
|  6        |  0.0      |  17.16    | -27.45    |  0.3117   |  170.7    |  878.3    |
|  7        |  0.0      |  16.05    | -27.84    |  0.5462   |  997.2    |  993.9    |
|  8        |  0.0      |  12.78    | -3.155    |  0.5466   |  1.496    |  977.0    |
|  9        |  0.0      |  15.58    | -27.74    |  0.3524   |  995.4    |  9.24     |
|  10       |  0.0      |  16.77    | -2.102    |  0.5967   |  2.982    |  12.13    |
|  11       |  0.0      |  13.0     | -1.156    |  0.4581   |  984.1    |  989.7    |
|  12       |  0.0      |  16.03    | -1.713    |  0.3

|  17       |  0.0      |  11.57    | -0.2884   |  0.3072   |  21.9     |  988.9    |
|  18       |  0.0      |  12.04    | -1.319    |  0.4916   |  8.188    |  27.39    |
|  19       |  0.0      |  11.35    | -7.496    |  0.3558   |  490.5    |  717.1    |
|  20       |  0.0      |  10.05    | -26.98    |  0.4373   |  971.0    |  994.4    |
{'target': 0.0, 'params': {'n_clusters': 13.336176037620593, 'silence_threshold': -8.390265196735257, 'sim_thresh': 0.30003431244520345, 'slice_length': 303.0302400592079, 'step_size_audio': 147.60913492629592}}
Running workflow audio-tfidf-cosine-median_(3,3)-spectral_clustering
|   iter    |  target   | n_clus... | silenc... | sim_th... | slice_... | step_s... |
-------------------------------------------------------------------------------------
|  1        |  0.0      |  14.5     | -8.39     |  0.3      |  303.0    |  147.6    |
|  2        |  0.0      |  12.55    | -24.41    |  0.4037   |  397.4    |  539.3    |
|  3        |  0.0      |  14.5

|  6        |  0.0      |  17.37    | -27.45    |  0.3117   |  170.7    |  878.3    |
|  7        |  0.0      |  16.54    | -27.84    |  0.5462   |  997.2    |  993.9    |
|  8        |  0.0      |  14.09    | -3.155    |  0.5466   |  1.496    |  977.0    |
|  9        |  0.0      |  16.18    | -27.74    |  0.3524   |  995.4    |  9.24     |
|  10       |  0.0      |  17.08    | -2.102    |  0.5967   |  2.982    |  12.13    |
|  11       |  0.0      |  14.25    | -1.156    |  0.4581   |  984.1    |  989.7    |
|  12       |  0.0      |  16.52    | -1.713    |  0.3766   |  983.2    |  2.261    |
|  13       |  0.0      |  12.71    | -29.56    |  0.4601   |  39.75    |  982.5    |
|  14       |  0.0      |  16.41    | -24.26    |  0.4545   |  8.301    |  19.18    |
|  15       |  0.0      |  12.52    | -0.168    |  0.4996   |  997.0    |  955.3    |
|  16       |  0.0      |  14.58    | -29.86    |  0.4871   |  988.5    |  16.3     |
|  17       |  0.0      |  13.18    | -0.2884   |  0.3

{'target': 0.0, 'params': {'n_clusters': 14.502132028215444, 'silence_threshold': -8.390265196735257, 'sim_thresh': 0.30003431244520345, 'slice_length': 303.0302400592079, 'step_size_audio': 147.60913492629592}}
running on video tORLeHHtazM
Running workflow audio-tfidf-cosine-None-spectral_clustering
|   iter    |  target   | n_clus... | silenc... | sim_th... | slice_... | step_s... |
-------------------------------------------------------------------------------------
|  1        |  0.0      |  14.5     | -8.39     |  0.3      |  303.0    |  147.6    |
|  2        |  0.0      |  12.55    | -24.41    |  0.4037   |  397.4    |  539.3    |
|  3        |  0.0      |  14.52    | -9.443    |  0.3613   |  878.2    |  28.36    |
|  4        |  0.0      |  16.02    | -17.48    |  0.4676   |  141.2    |  198.9    |
|  5        |  0.0      |  16.8     | -0.9522   |  0.394    |  692.6    |  876.5    |
|  6        |  0.0      |  17.37    | -27.45    |  0.3117   |  170.7    |  878.3    |
|  7      

Check the model on the Training set

In [30]:
df_results.to_csv('../data/processed/bayesian_opt/results.csv')

Examine bias and variance

In [9]:
''' Calculate bias'''
Ex = df_results.groupby(["workflow",'video'])['max_target'].mean()
Fx = 1
Ex =Ex.apply(lambda x: Fx-x)
Ex

workflow                                                      video      
sliding_window-tfidf-cosine-None-spectral_clustering          2mC1uqwEmWQ    0.573333
                                                              8BeXwhljq2g    0.733333
                                                              B-Xe7_mf2CY    0.788889
                                                              BiLJmhQMbVg    1.000000
                                                              MkiUBJcgdUY    0.722222
                                                              Q-HugPvA7GQ    0.828571
                                                              SCvtxjpVQms    0.953846
                                                              dkAr9ThdSUU    0.600000
                                                              ksh6OMRFroI    0.516667
                                                              tORLeHHtazM    1.000000
                                                              x5zL

In [12]:
'''Calculating variance'''
df_results.groupby(["workflow",'video'])['max_target'].var()

workflow                                                      video      
sliding_window-tfidf-cosine-None-spectral_clustering          2mC1uqwEmWQ    0.063556
                                                              8BeXwhljq2g    0.133333
                                                              B-Xe7_mf2CY    0.016049
                                                              BiLJmhQMbVg    0.000000
                                                              MkiUBJcgdUY    0.009259
                                                              Q-HugPvA7GQ    0.027041
                                                              SCvtxjpVQms    0.010651
                                                              dkAr9ThdSUU    0.050865
                                                              ksh6OMRFroI    0.195833
                                                              tORLeHHtazM    0.000000
                                                              x5zL